In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.util import Surv

# -------------------------
# 1. Load the tables
# -------------------------

from pathlib import Path

cwd = Path.cwd()
project_root = None

# Fallback: search for the data/raw/train directory structure
if project_root is None:
    for p in [cwd] + list(cwd.parents):
        if (p / 'data' / 'raw' / 'train').is_dir():
            project_root = p
            break

if project_root is None:
    project_root = cwd

data_raw_train = project_root / 'data' / 'raw' / 'train'
clinical_path = data_raw_train / 'clinical_train.csv'
molecular_path = data_raw_train / 'molecular_train.csv'
target_path = data_raw_train / 'target_train.csv'

print(f"Project root: {project_root}")
print(f"Looking for data at: {data_raw_train}")

if not target_path.exists():
    raise FileNotFoundError(f'Could not find target file at {target_path}')
if not clinical_path.exists():
    raise FileNotFoundError(f'Could not find clinical file at {clinical_path}')
if not molecular_path.exists():
    raise FileNotFoundError(f'Could not find molecular file at {molecular_path}')


target = pd.read_csv(target_path, sep=",", header=0,
                     names=['ID','OS_YEARS','OS_STATUS'],
                     dtype={'ID': str, 'OS_YEARS': float, 'OS_STATUS': float})

clinical = pd.read_csv(clinical_path, sep=",", header=0,
    names=['ID','CENTER','BM_BLAST','WBC','ANC','MONOCYTES','HB','PLT','CYTOGENETICS'],
    dtype={'ID': str, 'CENTER': str, 'BM_BLAST': float, 'WBC': float,
           'ANC': float, 'MONOCYTES': float, 'HB': float, 'PLT': float, 'CYTOGENETICS': str})

molecular = pd.read_csv(molecular_path, sep=",", header=0,
    names=['ID','CHR','START','END','REF','ALT','GENE','PROTEIN_CHANGE',
           'EFFECT','VAF','DEPTH'],
    dtype={'ID': str, 'CHR': str, 'START': float, 'END': float,
           'REF': str, 'ALT': str, 'GENE': str, 'PROTEIN_CHANGE': str,
           'EFFECT': str, 'VAF': float, 'DEPTH': float})

Project root: c:\Users\alexb\Documents\EI3\APST1\Data Challenge\project_root
Looking for data at: c:\Users\alexb\Documents\EI3\APST1\Data Challenge\project_root\data\raw\train


In [5]:
# We check that the data is loaded correctly
print(f"Target shape: {target.shape}")
print(f"Clinical shape: {clinical.shape}")
print(f"Molecular shape: {molecular.shape}")

Target shape: (3323, 3)
Clinical shape: (3323, 9)
Molecular shape: (10935, 11)


In [14]:
# -------------------------
# 2. Convert molecular table into per-patient mutation features
# -------------------------

# FLAG: Mutation feature engineering mode
#   0 = mutation count only
#   1 = lethal mutation flags (statistically significant)
#   2 = all mutations as binary indicators
mutation_feature_mode = 2

# List of genes to exclude from features (manual exclusions)
EXCLUDE_GENES = ["WT1", "ZBTB33"]

# Binary indicator for each gene mutated
gene_counts = molecular.groupby(['ID', 'GENE']).size().unstack(fill_value=0)

# Drop explicitly excluded genes if present
for g in EXCLUDE_GENES:
    if g in gene_counts.columns:
        gene_counts = gene_counts.drop(columns=g)

if mutation_feature_mode == 0:
    # Simple mutation count per patient: keep only TOTAL_MUTATED_GENES
    gene_counts["TOTAL_MUTATED_GENES"] = (gene_counts > 0).sum(axis=1)
    # Reduce to single column so no individual gene flags remain
    gene_counts = gene_counts[["TOTAL_MUTATED_GENES"]]
    print(f"Feature engineering with mutation count only:")
    print(f"  - Only TOTAL_MUTATED_GENES retained")

elif mutation_feature_mode == 1:
    # Define lethal mutations: top 10 genes by mortality rate from statistically significant mutations
    lethal_mutations = [g for g in mutation_df_filtered.head(10).index.tolist() if g not in EXCLUDE_GENES]
    print(f"Lethal mutations selected (statistically significant, p < 0.05) excluding manual list:")
    for gene in lethal_mutations:
        row = mutation_df_filtered.loc[gene]
        print(f"  - {gene}: {row['mortality_rate']:.1f}% ({int(row['deaths'])}/{int(row['patient_count'])} patients, p={row['p_value']:.4f})")
    
    # Ensure columns for lethal mutations exist and are binary
    for gene in lethal_mutations:
        if gene in gene_counts.columns:
            gene_counts[gene] = (gene_counts[gene] > 0).astype(int)
        else:
            gene_counts[gene] = 0

    # Aggregate all other (non-lethal) genes into OTHER_MUTATIONS_COUNT
    existing_cols = [c for c in gene_counts.columns if c not in lethal_mutations]
    if existing_cols:
        gene_counts["OTHER_MUTATIONS_COUNT"] = gene_counts[existing_cols].sum(axis=1).astype(int)
        # Drop the non-lethal individual gene columns
        gene_counts = gene_counts.drop(columns=existing_cols)

    # Now explicitly keep only lethal flags + OTHER_MUTATIONS_COUNT
    final_cols = lethal_mutations.copy()
    if 'OTHER_MUTATIONS_COUNT' in gene_counts.columns:
        final_cols.append('OTHER_MUTATIONS_COUNT')

    # Add TOTAL_MUTATED_GENES as sum of lethal flags + other count
    gene_counts["TOTAL_MUTATED_GENES"] = gene_counts[final_cols].sum(axis=1).astype(int)
    final_cols.append('TOTAL_MUTATED_GENES')

    # Reduce gene_counts to final columns only (enforces no stray gene flags)
    gene_counts = gene_counts[final_cols]

    print(f"\nFeature engineering with lethal mutation flags (strict):")
    print(f"  - Kept lethal flags: {len(lethal_mutations)}")
    print(f"  - Kept OTHER_MUTATIONS_COUNT: {'OTHER_MUTATIONS_COUNT' in gene_counts.columns}")
    print(f"  - Kept TOTAL_MUTATED_GENES: {'TOTAL_MUTATED_GENES' in gene_counts.columns}")

elif mutation_feature_mode == 2:
    # Keep all genes as binary indicators
    for col in gene_counts.columns:
        gene_counts[col] = (gene_counts[col] > 0).astype(int)
    
    # Adding a column with a total number of mutated genes
    gene_counts["TOTAL_MUTATED_GENES"] = (gene_counts > 0).sum(axis=1)
    
    print(f"Feature engineering without lethal mutation flags:")
    print(f"  - All {len(gene_counts.columns) - 1} genes included as binary indicators")

# VAF summary statistics:
vaf_stats = molecular.groupby("ID")["VAF"].agg(['mean','max','min']).add_prefix("VAF_")

# Combine molecular features
mol_features = gene_counts.join(vaf_stats, how="left").fillna(0)

# -------------------------
# 3. Merge everything into a single training table
# -------------------------

X = clinical.merge(mol_features, how="left", on="ID").fillna(0)

# Remove ID column
X = X.set_index("ID")

# Drop complex/unwanted clinical columns
if 'CYTOGENETICS' in X.columns:
    X = X.drop(columns=['CYTOGENETICS'])

# Force categorical fields to string (only keep CENTER if present)
if 'CENTER' in X.columns:
    X["CENTER"] = X["CENTER"].astype(str)

# Prepare survival data: need both OS_YEARS (time) and OS_STATUS (event indicator)
survival_data = target.set_index("ID").loc[X.index][["OS_YEARS", "OS_STATUS"]].copy()
survival_data["OS_STATUS"] = survival_data["OS_STATUS"].astype(bool)

# Remove patients with missing survival time
valid_idx = ~survival_data["OS_YEARS"].isna()
X = X[valid_idx]
survival_data = survival_data[valid_idx]

# Create structured array for survival analysis
y = Surv.from_arrays(
    event=survival_data["OS_STATUS"].values,
    time=survival_data["OS_YEARS"].values
)


Feature engineering without lethal mutation flags:
  - All 122 genes included as binary indicators


C:\Users\alexb\AppData\Local\Temp\ipykernel_2060\2630624795.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gene_counts["TOTAL_MUTATED_GENES"] = (gene_counts > 0).sum(axis=1)


In [15]:
# We check the integrity of the merged data
print(X)

        CENTER  BM_BLAST     WBC   ANC  MONOCYTES    HB    PLT  ABL1  ARID1A  \
ID                                                                             
P132697    MSK      14.0    2.80  0.20       0.70   7.6  119.0   0.0     0.0   
P132698    MSK       1.0    7.40  2.40       0.10  11.6   42.0   0.0     0.0   
P116889    MSK      15.0    3.70  2.10       0.10  14.2   81.0   0.0     0.0   
P132699    MSK       1.0    3.90  1.90       0.10   8.9   77.0   0.0     0.0   
P132700    MSK       6.0  128.00  9.70       0.90  11.1  195.0   0.0     0.0   
...        ...       ...     ...   ...        ...   ...    ...   ...     ...   
P121826     VU       1.0    2.50  1.02       0.20  10.2   78.0   0.0     0.0   
P121827     VU       1.5    8.10  2.66       0.45  11.3   40.0   0.0     1.0   
P121830     VU       0.0    1.80  0.55       0.29   9.4   86.0   0.0     0.0   
P121853     VU       5.0    1.37  0.37       0.11  11.4  102.0   0.0     0.0   
P121834     VU       0.0    2.70  0.72  

In [16]:
# -------------------------
# 4. Build preprocessing + model pipeline
# -------------------------

# Categorical columns from clinical features (compute dynamically in case CYTOGENETICS was dropped)
categorical_cols = [c for c in ["CENTER", "CYTOGENETICS"] if c in X.columns]
numeric_cols = [c for c in X.columns if c not in categorical_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)

# GradientBoostingSurvivalAnalysis - survival-aware model that properly handles censoring
gb = GradientBoostingSurvivalAnalysis(
    n_estimators=600,
    learning_rate=0.03,
    max_depth=3,
    subsample=0.8,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=42
)

pipeline = Pipeline([
    ("pre", preprocess),
    ("model", gb)
])


# -------------------------
# 5. Train/test split and training
# -------------------------

# Split data (must preserve survival structure)
idx_train, idx_test = train_test_split(range(len(X)), test_size=0.2, random_state=42)

X_train = X.iloc[idx_train]
X_test = X.iloc[idx_test]
y_train = y[idx_train]
y_test = y[idx_test]

# Fit the survival pipeline
pipeline.fit(X_train, y_train)

# Predict risk scores (higher = worse survival)
pred = pipeline.predict(X_test)

In [17]:
from lifelines.utils import concordance_index
from sksurv.metrics import concordance_index_ipcw

# lifelines expects higher scores => longer survival (predicted time),
# so invert risk scores for concordance.
c_index = concordance_index(
    event_times=y_test['time'],
    predicted_scores=-pred,               # <- invert
    event_observed=y_test['event'].astype(bool)
)
print("C-index:", c_index)

# -------------------------
# 6. Evaluating model performance
# -------------------------

# IPCW C-index accounts for censoring in the evaluation set
c_index_ipcw = concordance_index_ipcw(
    y_train,
    y_test,
    pred
)[0]   # first entry is the IPCW-c-index

print("IPCW C-index:", c_index_ipcw)

C-index: 0.7434307854878478
IPCW C-index: 0.6995201872194418


In [18]:
# -------------------------
# Hyperparameter Optimization
# -------------------------

from sklearn.model_selection import GridSearchCV

# Define hyperparameter grid for GradientBoostingSurvivalAnalysis
param_grid = {
    'model__n_estimators': [100, 300, 600],
    'model__learning_rate': [0.01, 0.03, 0.05],
    'model__max_depth': [2, 3, 4],
    'model__min_samples_split': [5, 10],
    'model__min_samples_leaf': [2, 3, 5],
    'model__subsample': [0.7, 0.8, 0.9],
    'model__max_features': [None, 'sqrt']
}

# Create GridSearchCV with survival-aware scoring
# Using negative concordance index for minimization (since GridSearchCV minimizes)
gs = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='neg_concordance_index_ipcw',  # requires custom scorer for survival
    n_jobs=-1,
    verbose=1
)

# Note: If using standard GridSearchCV without custom survival scorer,
# use a simpler parameter grid and manual cross-validation approach:
# For now, using RandomizedSearchCV for faster optimization with large grid

from sklearn.model_selection import RandomizedSearchCV
import numpy as np

rs = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_grid,
    n_iter=20,
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# Fit the random search
rs.fit(X_train, y_train)

# Extract best parameters
best_params = rs.best_params_

print("=" * 60)
print("Hyperparameter Optimization Results:")
print("=" * 60)
print(f"Best parameters found:")
for param, value in best_params.items():
    print(f"  {param}: {value}")
print(f"Best CV score: {rs.best_score_:.4f}")
print("=" * 60)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Hyperparameter Optimization Results:
Best parameters found:
  model__subsample: 0.9
  model__n_estimators: 300
  model__min_samples_split: 10
  model__min_samples_leaf: 5
  model__max_features: sqrt
  model__max_depth: 4
  model__learning_rate: 0.03
Best CV score: 0.7287
Hyperparameter Optimization Results:
Best parameters found:
  model__subsample: 0.9
  model__n_estimators: 300
  model__min_samples_split: 10
  model__min_samples_leaf: 5
  model__max_features: sqrt
  model__max_depth: 4
  model__learning_rate: 0.03
Best CV score: 0.7287


In [19]:
# -------------------------
# 7. Train Final Model with Best Hyperparameters
# -------------------------

# Create final model with best parameters (survival-aware)
gb_final = GradientBoostingSurvivalAnalysis(
    n_estimators=best_params['model__n_estimators'],
    learning_rate=best_params['model__learning_rate'],
    max_depth=best_params['model__max_depth'],
    min_samples_split=best_params['model__min_samples_split'],
    min_samples_leaf=best_params['model__min_samples_leaf'],
    subsample=best_params['model__subsample'],
    max_features=best_params['model__max_features'],
    random_state=42
)

# Create final pipeline
model = Pipeline([
    ("pre", preprocess),
    ("model", gb_final)
])

# Train final model on training data
model.fit(X_train, y_train)

# Make predictions on test data
pred = model.predict(X_test)

# Evaluate final model
c_index_final = concordance_index(
    event_times=y_test['time'],
    predicted_scores=-pred,
    event_observed=y_test['event'].astype(bool)
)

c_index_ipcw_final = concordance_index_ipcw(
    y_train,
    y_test,
    pred
)[0]

print("=" * 60)
print("Final Model Performance (with optimized hyperparameters):")
print("=" * 60)
print(f"C-index: {c_index_final:.4f}")
print(f"IPCW C-index: {c_index_ipcw_final:.4f}")
print("=" * 60)

Final Model Performance (with optimized hyperparameters):
C-index: 0.7381
IPCW C-index: 0.6908


In [ ]:
# -------------------------
# 8. Predict survivability score for all patients (using optimized model)
# -------------------------

survivability_score = model.predict(X)

# Attach score to IDs
output = pd.DataFrame({
    "ID": X.index,
    "SURVIVABILITY_SCORE": survivability_score
})

print(output.head())

# Save to CSV
output.to_csv("survivability_predictions.csv", index=False)